# Part 1: Experiment design

Question: We want to know what is the improvement in terms of number of rentals and the revenue generated when professional photos are used in a listing instead of non professional photos.

**Data to be selected:**

1. In order to determine this we can select all listings that have ONLY changed their photos to professional photos between 2016 and now. We want to make sure we don't select listings where the text has also been changed in order to avoid the influence of change in copy in the results. We select from 2016 because we want to know if it has had an influence recently.

2. For all these listings we can take 1 month of data before and after the change of pictures with data being the number of rentals and the revenue generated by these rentals. (Note that we only take 1 month to minimize the seasonality effect in the data. Indeed if we were to take 3 months we would increase the effect of comparing before and during summer for example, which has an influence itself on the price)

**Analysis of the results:**

Once we have collected the two sets of data we can plot the frequency distribution of before and after the change (with revenue and number of rentals) and check if the difference between the mean of the two is statistically significant. This can be done by performing a two sample t-test with a confidence interval of 95%.

# Part 2: Result analysis

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [ ]:
trips_raw = pd.read_json("../data/car_hailing_data.json")